## **Applio**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)

<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas: Hina (Encryption method), Poopmaster (Extra section), Shirou (UV installer)

#### **Disclaimer**
By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

### **Install Applio**
If the runtime restarts, re-run the installation steps.

In [ ]:
# @title Mount Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Setup runtime environment
from IPython.display import clear_output
import codecs

encoded_url = "uggcf://tvguho.pbz/VNUvfcnab/Nccyvb/"
decoded_url = codecs.decode(encoded_url, "rot_13")

repo_name_encoded = "Nccyvb"
repo_name = codecs.decode(repo_name_encoded, "rot_13")

LOGS_FOLDER = f"/content/{repo_name}/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/{repo_name}Backup"

%cd /content
!git config --global advice.detachedHead false
!git clone --depth 1 {decoded_url} --branch 3.2.9 --single-branch
%cd {repo_name}
clear_output()

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements.txt
!uv pip install -q ngrok
!npm install -g -q localtunnel &> /dev/null

!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"
print("✅ Finished installing requirements!")

### **Start Applio**

In [ ]:
# @title **Start server**
# @markdown  ### Choose a sharing method:
from os import environ
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = "If you selected the 'ngrok' method, obtain your auth token here: https://dashboard.ngrok.com/get-started/your-authtoken" # @param {type:"string"}
tensorboard = True #@param {type: "boolean"}

if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir logs --bind_all

match (method):
  case 'gradio':
    !python app.py --listen --share
  case 'localtunnel':
    !echo Password IP: $(curl --silent https://ipv4.icanhazip.com)
    !lt --port 6969 & python app.py --listen
  case 'ngrok':
    import ngrok
    ngrok.kill()
    listener = await ngrok.forward(6969, authtoken=ngrok_token)
    print(f"Ngrok URL: {listener.url()}")
    !python app.py --listen